# Setup

In [6]:
import numpy as np
from math import sqrt
import csv as commaSeparatedValues
# Inisiasi variabel-variabel
devices = ["C20", "Evan", "Alba"]
base_raw_directory = '/content/drive/My Drive/Capstone/Dirapikan/'
base_save_directory = '/content/drive/My Drive/Capstone/Spreadsheet/Bayesian Estimator Lintas Perangkat/'
# Indexing tingkat 1: device. Tingkat 3: Tx
rssi = np.empty([3, 6, 2, 6, 7, 200], dtype=int)
# Indexing tingkat 1: device. Tingkat 3: Tx. Tingkat 6: 0 = max, 1 = median, 2 = mean
rssi_centering = np.empty([3, 6, 2, 6, 7, 3], dtype=float)
# Indexing tingkat 1: device. Tingkat 2: Tx. Tingkat 3: data centering
centering_max = np.empty([3, 2, 3], dtype=float)
centering_min = np.empty([3, 2, 3], dtype=float)
# Indexing tingkat 3: data centering
bayesianLikelihood = np.empty([6, 7, 3], dtype=float)
# Variabel untuk keluaran ke .csv
fields = ['Distribusi Kekuatan Transmisi', 'Tipe Pusatan Data', 'Rerata Galat (meter)', 'Standar Deviasi Galat', 'ECDF 95% Galat (meter)']
row = np.empty(shape=(5), dtype='object')

In [2]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#@title Form Parameter
trainingRatio = 0.8 #@param {type:"slider", min:0.05, max:0.95, step:0.05}
trainingLength = int(trainingRatio*200)
distanceOrder = 2 #@param {type:"number"}
windowLength =  5#@param {tipe: "integer"}
# Indexing tingkat 4: data centering. Tingkat 5: 0 = ordinat perkiraan, 1 = axis perkiraan, 2 = galat (meter)
prediction = np.empty([6, 7, 3 * (201-trainingLength-windowLength), 3, 3], dtype=float)
# Indexing tingkat 6: data centering
rssi_normalized = np.empty([3, 6, 2, 6, 7, 3, trainingLength], dtype=float)

# Exhaustive Search Akurasi per Distribusi Daya Transmisi

In [10]:
# Kalau .csv-nya belum created aja
with open(base_save_directory + "Mixed 80-20 Power " + str(distanceOrder) + " Window " + str(windowLength) + ".csv", 'w', newline='') as csvFile:
    csvWriter = commaSeparatedValues.writer(csvFile)
    csvWriter.writerow(fields)
    csvFile.close()

In [11]:
# Definisi fungsi pembantu
def normalize(ndarray, device, tx, centering_type):
    return (ndarray - centering_min[device, tx, centering_type])/(centering_max[device, tx, centering_type] - centering_min[device, tx, centering_type])
def search(tx):
    for axis in range(7):
        for ordinate in range(6):
            test_number = 0
            for device in range(3):
                for device_test_number in range(trainingLength, 201-windowLength):
                    for calc_axis in range(7):
                        for calc_ordinate in range(6):
                            rp_data = np.empty([6], dtype=float)
                            test_data = np.empty([6], dtype=float)
                            distance_power = np.empty([3], dtype=float)
                            var = np.zeros([3], dtype=float)
                            # Max
                            for beacon in range(6):
                                rp_data[beacon] = rssi_centering[0, beacon, tx[beacon], calc_ordinate, calc_axis, 0]
                                test_data[beacon] = np.max(normalize(rssi[device, beacon, tx[beacon], ordinate, axis, device_test_number:windowLength+device_test_number], device, tx[beacon], 0))
                                var[0] += np.var(rssi_normalized[:, beacon, tx[beacon], ordinate, axis, 0, :])
                            distance_power[0] = np.sum(abs(rp_data-test_data)**distanceOrder)/6
                            var[0] /= 6
                            # Median
                            for beacon in range(6):
                                rp_data[beacon] = rssi_centering[0, beacon, tx[beacon], calc_ordinate, calc_axis, 1]
                                test_data[beacon] = np.median(normalize(rssi[device, beacon, tx[beacon], ordinate, axis, device_test_number:windowLength+device_test_number], device, tx[beacon], 1))
                                var[1] += np.var(rssi_normalized[:, beacon, tx[beacon], ordinate, axis, 1, :])
                            distance_power[1] = np.sum(abs(rp_data-test_data)**distanceOrder)/6
                            var[1] /= 6
                            # Mean
                            for beacon in range(6):
                                rp_data[beacon] = rssi_centering[0, beacon, tx[beacon], calc_ordinate, calc_axis, 2]
                                test_data[beacon] = np.mean(normalize(rssi[device, beacon, tx[beacon], ordinate, axis, device_test_number:windowLength+device_test_number], device, tx[beacon], 2))
                                var[2] += np.var(rssi_normalized[:, beacon, tx[beacon], ordinate, axis, 2, :])
                            distance_power[2] = np.sum(abs(rp_data-test_data)**distanceOrder)/6
                            var[2] /= 6
                            bayesianLikelihood[calc_ordinate, calc_axis, :] = np.exp(np.divide(-0.5*distance_power**(2/distanceOrder), var))
                    # Max
                    coordinate = np.unravel_index(np.argmax(bayesianLikelihood[:, :, 0]), (6, 7))
                    prediction[ordinate, axis, test_number, 0, 0] = coordinate[0]
                    prediction[ordinate, axis, test_number, 0, 1] = coordinate[1]
                    prediction[ordinate, axis, test_number, 0, 2] = sqrt((coordinate[0]-ordinate)**2 + (coordinate[1]-axis)**2)
                    # Median
                    coordinate = np.unravel_index(np.argmax(bayesianLikelihood[:, :, 1]), (6, 7))
                    prediction[ordinate, axis, test_number, 1, 0] = coordinate[0]
                    prediction[ordinate, axis, test_number, 1, 1] = coordinate[1]
                    prediction[ordinate, axis, test_number, 1, 2] = sqrt((coordinate[0]-ordinate)**2 + (coordinate[1]-axis)**2)
                    # Mean
                    coordinate = np.unravel_index(np.argmax(bayesianLikelihood[:, :, 2]), (6, 7))
                    prediction[ordinate, axis, test_number, 2, 0] = coordinate[0]
                    prediction[ordinate, axis, test_number, 2, 1] = coordinate[1]
                    prediction[ordinate, axis, test_number, 2, 2] = sqrt((coordinate[0]-ordinate)**2 + (coordinate[1]-axis)**2)
                    test_number += 1
def tx_to_str(tx):
    if(tx == 0):
        return "-4"
    elif(tx == 1):
        return "-12"
    else:
        return "error"
# Membaca csv dan pemusatan data training
for device in range(3):
    for axis in range(7):
        for ordinate in range(6):
            csv4 = np.genfromtxt(base_raw_directory + devices[device] + '/' + str(axis) + ',' + str(ordinate) + "_" + devices[device] + ".csv", delimiter=',', case_sensitive=False)
            csv12 = np.genfromtxt(base_raw_directory + devices[device] + '-12/' + str(axis) + ',' + str(ordinate) + "_" + devices[device] + "-12.csv", delimiter=',', case_sensitive=False)
            for tx in range(2):
                for beacon in range(6):
                    if tx == 0:
                        rssi[device, beacon, 0, ordinate, axis, :] = csv4[beacon,5:205]
                    else:
                        rssi[device, beacon, 1, ordinate, axis, :] = csv12[beacon,5:205]
                    rssi_centering[device, beacon, tx, ordinate, axis, 0] = np.max(rssi[device, beacon, tx, ordinate, axis, 0:trainingLength])
                    rssi_centering[device, beacon, tx, ordinate, axis, 1] = np.median(rssi[device, beacon, tx, ordinate, axis, 0:trainingLength])
                    rssi_centering[device, beacon, tx, ordinate, axis, 2] = np.mean(rssi[device, beacon, tx, ordinate, axis, 0:trainingLength])
    for tx in range(2):
        for centering_type in range (3):
            centering_max[device, tx, centering_type] = np.max(rssi_centering[device, :, tx, :, :, centering_type])
            centering_min[device, tx, centering_type] = np.min(rssi_centering[device, :, tx, :, :, centering_type])
            rssi_normalized[device, :, tx, :, :, centering_type, :] = normalize(rssi[device, :, tx, :, :, 0:trainingLength], device, tx, centering_type)
for tx in range(2):
    for axis in range(7):
        for ordinate in range(6):
            for beacon in range(6):
                rssi_centering[0, beacon, tx, ordinate, axis, 0] = np.max(rssi_normalized[:, beacon, tx, ordinate, axis, 0, :])
                rssi_centering[0, beacon, tx, ordinate, axis, 1] = np.median(rssi_normalized[:, beacon, tx, ordinate, axis, 1, :])
                rssi_centering[0, beacon, tx, ordinate, axis, 2] = np.mean(rssi_normalized[:, beacon, tx, ordinate, axis, 2, :])
# Mengukur rerata dan persentil 95 galat pada setiap kombinasi distribusi daya dan pemusatan data
counter = 0
for tx1 in range(2):
    for tx2 in range(2):
        for tx3 in range(2):
            for tx4 in range(2):
                for tx5 in range(2):
                    for tx6 in range(2):
                        print("calculating " + tx_to_str(tx1) + ", " + tx_to_str(tx2) + ", " + tx_to_str(tx3) + ", " + tx_to_str(tx4) + ", " + tx_to_str(tx5) + ", " + tx_to_str(tx6))
                        search([tx1, tx2, tx3, tx4, tx5, tx6])
                        for centeringType in range(3):
                            row[0] = tx_to_str(tx1) + ", " + tx_to_str(tx2) + ", " + tx_to_str(tx3) + ", " + tx_to_str(tx4) + ", " + tx_to_str(tx5) + ", " + tx_to_str(tx6)
                            if(centeringType == 0):
                                row[1] = "Max"
                            elif(centeringType == 1):
                                row[1] = "Median"
                            elif(centeringType == 2):
                                row[1] = "Mean"
                            else:
                                row[1] = "Error"
                            row[2] = np.mean(prediction[:, :, :, centeringType, 2])
                            row[3] = np.std(prediction[:, :, :, centeringType, 2])
                            row[4] = np.percentile(prediction[:, :, :, centeringType, 2], 95)
                            # Append keluaran ke .csv
                            with open(base_save_directory + "Mixed 80-20 Power " + str(distanceOrder) + " Window " + str(windowLength) + ".csv", 'a', newline='') as csvFile:
                                csvWriter = commaSeparatedValues.writer(csvFile)
                                csvWriter.writerow(row)
                                csvFile.close()
                            print(counter)
                            counter += 1

calculating -4, -4, -4, -4, -4, -4
0
1
2
calculating -4, -4, -4, -4, -4, -12
3
4
5
calculating -4, -4, -4, -4, -12, -4


KeyboardInterrupt: ignored

In [12]:
drive.flush_and_unmount()